In [1]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 7.7 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=37c59e3efe3f7ad200504dbd740eb09f6d2d9303830929c07591d42ee1deebc0
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


Loading main data

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
data = yf.download("SBIN.NS" , start = "2018-01-01" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [4]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-01,310.600006,312.750000,306.299988,307.100006,304.290649,12182400
2018-01-02,309.000000,309.000000,301.350006,303.250000,300.475861,14797364
2018-01-03,305.299988,306.450012,302.149994,302.850006,300.079529,13857373


In [5]:
# Sort the data points based on indexes just for confirmation 
data.sort_index(inplace = True)

In [6]:
# Remove any duplicate index 
data = data.loc[~data.index.duplicated(keep='first')]

In [7]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-09-20,449.399994,453.500000,435.649994,437.450012,437.450012,18376916
2021-09-21,440.149994,441.350006,432.649994,438.799988,438.799988,17279588
2021-09-22,438.899994,442.799988,436.100006,438.850006,438.850006,14317153


In [8]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-01,310.600006,312.750000,306.299988,307.100006,304.290649,12182400
2018-01-02,309.000000,309.000000,301.350006,303.250000,300.475861,14797364
2018-01-03,305.299988,306.450012,302.149994,302.850006,300.079529,13857373
2018-01-04,304.500000,310.200012,303.149994,308.500000,305.677826,24517099
2018-01-05,311.399994,311.399994,305.600006,306.350006,303.547485,14580575


In [9]:
# Check for missing values 
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [10]:
# Get the statistics of the data
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,917.000000,917.000000,917.000000,917.000000,917.000000,9.170000e+02
mean,292.675627,296.813087,287.983751,292.274318,289.929812,3.812468e+07
std,69.448503,69.866048,68.919613,69.437086,69.455560,2.683301e+07
min,151.949997,153.199997,149.449997,150.850006,149.470016,3.312160e+06
25%,253.000000,258.000000,248.550003,253.800003,251.478226,1.893280e+07
50%,288.700012,292.399994,283.799988,287.450012,284.820404,3.044297e+07
75%,331.700012,335.750000,325.850006,331.149994,328.120605,4.923498e+07
max,468.000000,471.899994,450.350006,463.700012,463.700012,2.149557e+08


Understanding Trends with in the Data

In [11]:
import plotly.graph_objects as go

In [12]:
# Check the trend in Closing Values 
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [13]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Volume')
fig.show()

Data Preparation

In [14]:
from sklearn.preprocessing import MinMaxScaler 
import pickle 
from tqdm.notebook import tnrange

In [15]:
# Filter only required data 
data = data[['Close' , 'Volume']]
data.head(3)

,Close,Volume
Date,,
2018-01-01,307.100006,12182400
2018-01-02,303.250000,14797364
2018-01-03,302.850006,13857373


Scrapping extra information

In [16]:
import requests 

response = requests.get('https://www.alphavantage.co/query?function=RSI&symbol=GOOGL&interval=daily&time_period=5&series_type=close&apikey=43T9T17VCV2ME4SM') 
response = response.json()

In [17]:
response.keys()

dict_keys(['Meta Data', 'Technical Analysis: RSI'])

In [18]:
rsi_data = pd.DataFrame.from_dict(response['Technical Analysis: RSI'] , orient='index')

In [19]:
rsi_data.head()

,RSI
2004-08-26,72.7683
2004-08-27,64.2686
2004-08-30,47.8388
2004-08-31,49.2490
2004-09-01,41.0745


In [20]:
rsi_data = rsi_data[rsi_data.index >= '2018-01-01']

In [21]:
rsi_data['RSI'] = rsi_data['RSI'].astype(np.float64)

In [22]:
rsi_data.head()

,RSI
2018-01-02,65.4310
2018-01-03,78.3686
2018-01-04,80.4830
2018-01-05,86.2429
2018-01-08,87.4882


In [23]:
data = data.merge(rsi_data, left_index=True, right_index=True, how='inner')

In [24]:
data.head()

,Close,Volume,RSI
2018-01-02,303.250000,14797364,65.4310
2018-01-03,302.850006,13857373,78.3686
2018-01-04,308.500000,24517099,80.4830
2018-01-05,306.350006,14580575,86.2429
2018-01-08,305.799988,13557197,87.4882


In [25]:
# Confirm the Testing Set length 
test_length = data[(data.index >= '2020-09-01')].shape[0]

In [26]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length): 
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [27]:
X , Y = CreateFeatures_and_Targets(data , 32)

  0%|          | 0/858 [00:00<?, ?it/s]

In [28]:
# Check the shapes
X.shape , Y.shape

((858, 32, 3), (858,))

In [29]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [30]:
# Check Training Dataset Shape 
Xtrain.shape , Ytrain.shape

((604, 32, 3), (604,))

In [31]:
# Check Testing Dataset Shape
Xtest.shape , Ytest.shape

((254, 32, 3), (254,))

In [32]:
# Create a Scaler to Scale Vectors with Multiple Dimensions 
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:,:,i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X 

In [33]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [34]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [35]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    data = pickle.load(pickle_in)
    return data

In [36]:
# Save your objects for future purposes 
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

Model Building

In [37]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=5, min_lr=0.00001,verbose = 1)

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional , BatchNormalization

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 3))))
model.add(LSTM(256 ,recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [39]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [40]:
history = model.fit(Xtrain, Ytrain,
            epochs=10,
            batch_size = 1,
            verbose=1,
            shuffle=False ,
            validation_data=(Xtest , Ytest),
            callbacks=[reduce_lr , save_best])

Epoch 1/10
604/604 [==============================] - 146s 229ms/step - loss: 0.0151 - val_loss: 0.4776
Epoch 2/10
604/604 [==============================] - 139s 231ms/step - loss: 0.0102 - val_loss: 0.4303
Epoch 3/10
604/604 [==============================] - 142s 235ms/step - loss: 0.0101 - val_loss: 0.4233
Epoch 4/10
604/604 [==============================] - 138s 229ms/step - loss: 0.0088 - val_loss: 0.3946
Epoch 5/10
604/604 [==============================] - 137s 228ms/step - loss: 0.0084 - val_loss: 0.3555
Epoch 6/10
604/604 [==============================] - 138s 229ms/step - loss: 0.0084 - val_loss: 0.3460
Epoch 7/10
604/604 [==============================] - 137s 227ms/step - loss: 0.0072 - val_loss: 0.3121
Epoch 8/10
604/604 [==============================] - 138s 228ms/step - loss: 0.0068 - val_loss: 0.2985
Epoch 9/10
604/604 [==============================] - 137s 227ms/step - loss: 0.0074 - val_loss: 0.2691
Epoch 10/10
604/604 [==============================] - 140s 231m

In [41]:
# Load the best weights
model.load_weights("best_weights.h5")

Visualize prediction on Test Set

In [42]:
Predictions = model.predict(Xtest)

In [43]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [44]:
Predictions.shape

(254, 1)

In [45]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [46]:
# Check the Predictions vs Actual
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index[-test_length:] , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index[-test_length:] , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

Visualize Prediction on whole data

In [47]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [48]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [49]:
Predictions = model.predict(Total_features)

In [50]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [51]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [52]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

In [53]:
# Save and Load the whole model
model.save("Model.h5")
loaded_model = tf.keras.models.load_model("Model.h5")

Realtime Prediction

In [54]:
def PredictStockPrice(Model , DataFrame , PreviousDate , feature_length = 32):
    idx_location = DataFrame.index.get_loc(PreviousDate)
    Features = DataFrame.iloc[idx_location - feature_length : idx_location,:].values
    Features = np.expand_dims(Features , axis = 0)
    Features = Feature_Scaler.transform(Features)
    Prediction = Model.predict(Features)
    Prediction = Target_Scaler.inverse_transform(Prediction)
    return Prediction[0][0]

In [55]:
PredictStockPrice(loaded_model , data , '2021-01-14')

230.20613